# **GoogleNet**
此份程式碼會介紹如何使用 tf.keras 的方式建構 GoogleNet 的模型架構，以及訓練的方式。

<img src="https://i.imgur.com/g5fRr89.png" width=1000/>

- [source paper](https://arxiv.org/abs/1409.4842)

## 匯入套件

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Tensorflow 相關套件
import tensorflow as tf
from tensorflow.keras import datasets, layers, Model, Sequential, losses

## 載入資料集

In [ ]:
(x_train, y_train), (x_test, y_test) = datasets.mnist.load_data()

# Expand dimensions
x_train = tf.expand_dims(x_train, axis=3, name=None)
x_test = tf.expand_dims(x_test, axis=3, name=None)
print(f'x_train shape: {x_train.shape}')
print(f'x_test shape: {x_test.shape}')
print('----------')

# Grayscale to RGB
x_train = tf.repeat(x_train, 3, axis=3)
x_test = tf.repeat(x_test, 3, axis=3)
print(f'x_train shape: {x_train.shape}')
print(f'x_test shape: {x_test.shape}')
print('----------')

# Split dataset into training and validation data
x_val = x_train[int(x_train.shape[0]*0.8):, :, :, :]
y_val = y_train[int(y_train.shape[0]*0.8):]
x_train = x_train[:int(x_train.shape[0]*0.8), :, :, :]
y_train = y_train[:int(y_train.shape[0]*0.8)]
print(f'x_train shape: {x_train.shape}, x_val shape: {x_val.shape}')
print(f'y_train shape: {y_train.shape}, y_val shape: {y_val.shape}')

## GoogleNet Architecture

<img src="https://i.imgur.com/4xLmfqx.png" width=1000/>

- [source paper](https://arxiv.org/abs/1409.4842)

In [ ]:
def inception_block(x, filters_1x1,
                    filters_3x3_reduce, filters_3x3,
                    filters_5x5_reduce, filters_5x5,
                    filters_pooling):

    path_1 = layers.Conv2D(filters_1x1, (1, 1),
                           padding='same',
                           activation='relu')(x)

    path_2 = layers.Conv2D(filters_3x3_reduce, (1, 1),
                           padding='same',
                           activation='relu')(x)
    path_2 = layers.Conv2D(filters_3x3, (3, 3),
                           padding='same',
                           activation='relu')(path_2)

    path_3 = layers.Conv2D(filters_5x5_reduce, (1, 1),
                           padding='same',
                           activation='relu')(x)
    path_3 = layers.Conv2D(filters_5x5, (5, 5),
                           padding='same',
                           activation='relu')(path_3)

    path_4 = layers.MaxPool2D((3, 3),
                              strides=(1, 1),
                              padding='same')(x)
    path_4 = layers.Conv2D(filters_pooling, (1, 1),
                           padding='same',
                           activation='relu')(path_4)

    return tf.concat([path_1, path_2, path_3, path_4], axis=3)

In [ ]:
def auxiliary_classifier(inputs):
    aux = layers.AveragePooling2D((5, 5), strides=3)(inputs)
    aux = layers.Conv2D(128, 1, padding='same', activation='relu')(aux)
    aux = layers.Flatten()(aux)
    aux = layers.Dense(1024, activation='relu')(aux)
    aux = layers.Dropout(0.7)(aux)
    outputs = layers.Dense(10, activation='softmax')(aux)
  
    return outputs

In [ ]:
labels_num = 10

In [ ]:
tf.keras.backend.clear_session()
inputs = layers.Input(shape=x_train.shape[1:])
x = layers.Resizing(224, 224,
                    interpolation="bilinear",
                    input_shape=x_train.shape[1:])(inputs)
x = layers.Conv2D(64, (7, 7), strides=2,
                  padding='same',
                  activation='relu')(x)
x = layers.MaxPooling2D((3, 3), strides=2)(x)
x = layers.Conv2D(64, (1, 1), strides=1,
                  padding='same',
                  activation='relu')(x)
x = layers.Conv2D(192, (3, 3), strides=1,
                  padding='same',
                  activation='relu')(x)
x = layers.MaxPooling2D((3, 3), strides=2)(x)

inception_3a = inception_block(x, filters_1x1=64,
                filters_3x3_reduce=96, filters_3x3=128,
                filters_5x5_reduce=16, filters_5x5=32,
                filters_pooling=32)

inception_3b = inception_block(inception_3a, filters_1x1=128,
                filters_3x3_reduce=128, filters_3x3=192,
                filters_5x5_reduce=32, filters_5x5=96,
                filters_pooling=64)
inception_3b = layers.MaxPooling2D((3, 3), strides=2)(inception_3b)

inception_4a = inception_block(inception_3b, filters_1x1=192,
                filters_3x3_reduce=96, filters_3x3=208,
                filters_5x5_reduce=16, filters_5x5=48,
                filters_pooling=64)

aux_1 = auxiliary_classifier(inception_4a)

inception_4b = inception_block(inception_4a, filters_1x1=160,
                 filters_3x3_reduce=112, filters_3x3=224,
                 filters_5x5_reduce=24, filters_5x5=64,
                 filters_pooling=64)

inception_4c = inception_block(inception_4b, filters_1x1=128,
                 filters_3x3_reduce=128, filters_3x3=256,
                 filters_5x5_reduce=24, filters_5x5=64,
                 filters_pooling=64)

inception_4d = inception_block(inception_4c, filters_1x1=112,
                 filters_3x3_reduce=144, filters_3x3=288,
                 filters_5x5_reduce=32, filters_5x5=64,
                 filters_pooling=64)

aux_2 = auxiliary_classifier(inception_4d)

inception_4e = inception_block(inception_4d, filters_1x1=256,
                 filters_3x3_reduce=160, filters_3x3=320,
                 filters_5x5_reduce=32, filters_5x5=128,
                 filters_pooling=128)

inception_4e = layers.MaxPooling2D((3, 3), strides=2)(inception_4e)

inception_5a = inception_block(inception_4e, filters_1x1=256,
                 filters_3x3_reduce=160, filters_3x3=320,
                 filters_5x5_reduce=32, filters_5x5=128,
                 filters_pooling=128)

inception_5b = inception_block(inception_5a, filters_1x1=384,
                 filters_3x3_reduce=192, filters_3x3=384,
                 filters_5x5_reduce=48, filters_5x5=128,
                 filters_pooling=128)

outputs = layers.GlobalAveragePooling2D()(inception_5b)
outputs = layers.Dropout(0.4)(outputs)
outputs = layers.Dense(labels_num,
                       activation='softmax')(outputs)

In [ ]:
GoogleNet_model = Model(inputs=inputs, outputs=[outputs, aux_1, aux_2])

In [ ]:
GoogleNet_model.summary()

In [ ]:
batch_size = 256
epochs = 10

In [ ]:
inputs = np.ones((batch_size, x_train.shape[1], x_train.shape[2], 3),
                 dtype=np.float32)
GoogleNet_model(inputs)

In [ ]:
GoogleNet_model.compile(
    optimizer='adam',
    loss=[losses.sparse_categorical_crossentropy,
          losses.sparse_categorical_crossentropy,
          losses.sparse_categorical_crossentropy],
    loss_weights=[1, 0.2, 0.2],
    metrics=['accuracy'])

history = GoogleNet_model.fit(x_train, y_train,
                              validation_data=(x_val, y_val),
                              batch_size=batch_size,
                              epochs=epochs)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(15,6))

axs[0].plot(history.history['loss'])
axs[0].plot(history.history['val_loss'])
axs[0].title.set_text('Training Loss vs Validation Loss')
axs[0].set_xlabel('Epochs')
axs[0].set_ylabel('Loss')
axs[0].legend(['Train', 'Val'])

axs[1].plot(history.history['dense_4_accuracy'])
axs[1].plot(history.history['val_dense_4_accuracy'])
axs[1].title.set_text('Training Accuracy vs Validation Accuracy')
axs[1].set_xlabel('Epochs')
axs[1].set_ylabel('Accuracy')
axs[1].legend(['Train', 'Val'])

In [ ]:
GoogleNet_model.evaluate(x_test, y_test)